# 모델 만드는 순서
1. 문제 정의 (어떤 문제 해결할건지)
2. 데이터 준비
  - 데이터 분할
3. 텍스트 전처리
  - 토큰화
  - 불용어 제거
  - 정규화
4. 숫자 벡터화
  - TF-IDF
5. 데이터셋 생성
6. 모델 설계
7. 모델 학습 : 데이터를 이용해 학습
8. 모델 평가 : 검증 데이터로 성능 평가
9. 모델 예측 및 활용

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/멋쟁이사자차럼/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

# 네이버 영화 리뷰 데이터
- 학습데이터
    - https://drive.google.com/file/d/1B1TjJQPR2POtZmUxUC7yjo6SqcL200D3/view?usp=sharing
- 테스트데이터
    - https://drive.google.com/file/d/1EsqnKZ-UWELNq46UPJ8psoOEc4nZNKDK/view?usp=sharing
- 긍정1, 부정0

In [6]:
train = pd.read_csv(f'{DATA_PATH}review_train.csv')
test = pd.read_csv(f'{DATA_PATH}review_test.csv')

In [7]:
train

,id,review,target
0,train_0,이런 최고의 영화를 이제서야 보다니,1
1,train_1,안봤지만 유승준나와서 비추.,0
2,train_2,시대를 못 따라간 연출과 촌스러운 영상미.,0
3,train_3,원소전 굿,1
4,train_4,ㅋㅋㅋㅋ 개봉영화평점단사람이1명 ㅋㅋㅋㅋ,1
...,...,...,...
1995,train_1995,사랑하고싶은 영화♥ 바위같은 편견 사이로 스며드는 빗물같은 영화,1
1996,train_1996,0점은 줄수없구나...,0
1997,train_1997,진짜재미있어요 강추 차태현 꺄 봉필,1
1998,train_1998,잠입자 보다 재밌는데?,1


# kiwi를 이용한 형태소 분석
- Kiwipiepy는 한국어 형태소 분석기인 Kiwi(Korean Intelligent Word Identifier)의 Python 라이브러리
- 품사 정보
    - https://github.com/bab2min/Kiwi#%ED%92%88%EC%82%AC-%ED%83%9C%EA%B7%B8

In [8]:
%pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.7/34.7 MB 32.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 66.2 MB/s eta 0:00:00
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.20.0-py3-none-any.whl size=34818026 sha256=9fbceaf7689f53bd53177b6ff8c4cc03c956300da67cd8e8215789e3b4a13121
  Stored in directory: /root/.cache/pip/wheels/b6/b1/66/2be9840f8ef3627d63d93503d81a5e3b41e9498dcb63b00b13
Successfully built kiwipiepy-model


In [9]:
from kiwipiepy import Kiwi # 클래스다!
kiwi = Kiwi() # 클래스니깐 객체 수행

In [10]:
text = train['review'][0]
text

'이런 최고의 영화를 이제서야 보다니'

## analyze 메서드
- 입력 문장을 형태소 단위로 분해하고, 각 형태소의 품사 및 세부 정보를 반환

- form: 형태소의 원본 형태 (예: "이런", "최고")
- tag: 품사 태그 (예: MM → 관형사, NNG → 일반명사)
- start: 형태소의 시작 인덱스
- len: 형태소의 길이
- score: 분석 결과의 신뢰도를 나타내는 점수

In [11]:
kiwi.analyze(text) # 토큰화!! # top_n=1 이 기본값

[([Token(form='이런', tag='MM', start=0, len=2),
   Token(form='최고', tag='NNG', start=3, len=2),
   Token(form='의', tag='JKG', start=5, len=1),
   Token(form='영화', tag='NNG', start=7, len=2),
   Token(form='를', tag='JKO', start=9, len=1),
   Token(form='이제서야', tag='MAG', start=11, len=4),
   Token(form='보', tag='VV', start=16, len=1),
   Token(form='다니', tag='EF', start=17, len=2)],
  -63.7940559387207)]

In [12]:
kiwi.analyze(text, top_n=2) # top_n 조정하여 여러개 결과 출력

[([Token(form='이런', tag='MM', start=0, len=2),
   Token(form='최고', tag='NNG', start=3, len=2),
   Token(form='의', tag='JKG', start=5, len=1),
   Token(form='영화', tag='NNG', start=7, len=2),
   Token(form='를', tag='JKO', start=9, len=1),
   Token(form='이제서야', tag='MAG', start=11, len=4),
   Token(form='보', tag='VV', start=16, len=1),
   Token(form='다니', tag='EF', start=17, len=2)],
  -63.7940559387207),
 ([Token(form='이런', tag='MM', start=0, len=2),
   Token(form='최고', tag='NNG', start=3, len=2),
   Token(form='의', tag='JKG', start=5, len=1),
   Token(form='영화', tag='NNG', start=7, len=2),
   Token(form='를', tag='JKO', start=9, len=1),
   Token(form='이제', tag='NNG', start=11, len=2),
   Token(form='서', tag='JKB', start=13, len=1),
   Token(form='야', tag='JX', start=14, len=1),
   Token(form='보', tag='VV', start=16, len=1),
   Token(form='다니', tag='EC', start=17, len=2)],
  -64.6888656616211)]

In [13]:
result = kiwi.analyze(text)
result[0][0] # = kiwi.tokenize(text)

[Token(form='이런', tag='MM', start=0, len=2),
 Token(form='최고', tag='NNG', start=3, len=2),
 Token(form='의', tag='JKG', start=5, len=1),
 Token(form='영화', tag='NNG', start=7, len=2),
 Token(form='를', tag='JKO', start=9, len=1),
 Token(form='이제서야', tag='MAG', start=11, len=4),
 Token(form='보', tag='VV', start=16, len=1),
 Token(form='다니', tag='EF', start=17, len=2)]

In [14]:
result[0]

([Token(form='이런', tag='MM', start=0, len=2),
  Token(form='최고', tag='NNG', start=3, len=2),
  Token(form='의', tag='JKG', start=5, len=1),
  Token(form='영화', tag='NNG', start=7, len=2),
  Token(form='를', tag='JKO', start=9, len=1),
  Token(form='이제서야', tag='MAG', start=11, len=4),
  Token(form='보', tag='VV', start=16, len=1),
  Token(form='다니', tag='EF', start=17, len=2)],
 -63.7940559387207)

- result 형태
```
result = [
  ([Token, Token, Token, ...], score),  # 첫 번째 분석 결과
  ([Token, Token, Token, ...], score),  # 두 번째 분석 결과 (top_n > 1일 경우)
  ...
]
```
```
result[0] = ([Token, Token, ...], score)
```
```
result[0][0] = [Token(form='이런', tag='MM', start=0, len=2),
                Token(form='최고', tag='NNG', start=3, len=2),
                Token(form='의', tag='JKG', start=5, len=1),
                Token(form='영화', tag='NNG', start=7, len=2),
                ...]
```

## tokenize 메서드
- 하나의 결과만 반환 (최고 확률의 분석 결과).
- analyze와 다르게 신뢰도 점수는 반환하지 않음
- 불필요한 세부 정보를 제외하고 간단한 형태소 분석만을 제공

In [15]:
result = kiwi.tokenize(text)
result

[Token(form='이런', tag='MM', start=0, len=2),
 Token(form='최고', tag='NNG', start=3, len=2),
 Token(form='의', tag='JKG', start=5, len=1),
 Token(form='영화', tag='NNG', start=7, len=2),
 Token(form='를', tag='JKO', start=9, len=1),
 Token(form='이제서야', tag='MAG', start=11, len=4),
 Token(form='보', tag='VV', start=16, len=1),
 Token(form='다니', tag='EF', start=17, len=2)]

In [16]:
len(result)

8

In [17]:
result[0].form # 토큰 문자열

'이런'

In [18]:
result[0].tag # 품사 문자열

'MM'

- iterable 한 객체를 전달할 경우 map 객체가 반환
  - iterable한 객체는 반복할 수 있는 객체를 의미
  - map 객체
    - 결과를 순차적으로 반환하는 객체
    - kiwi.tokenize()가 iterable을 입력받으면, 내부적으로 map 함수처럼 동작
    - 이때 반환되는 map 객체는 for 문이나 list()로 감싸서 접근해야 실제 결과를 볼 수 있습니다.

In [19]:
# 문장 2개
result = kiwi.tokenize(train['review'].iloc[:2]) # 맵 객체가 뭐지..?

for tokens in result:
  print(tokens)

[Token(form='이런', tag='MM', start=0, len=2), Token(form='최고', tag='NNG', start=3, len=2), Token(form='의', tag='JKG', start=5, len=1), Token(form='영화', tag='NNG', start=7, len=2), Token(form='를', tag='JKO', start=9, len=1), Token(form='이제서야', tag='MAG', start=11, len=4), Token(form='보', tag='VV', start=16, len=1), Token(form='다니', tag='EF', start=17, len=2)]
[Token(form='안', tag='MAG', start=0, len=1), Token(form='보', tag='VV', start=1, len=1), Token(form='었', tag='EP', start=1, len=1), Token(form='지만', tag='EC', start=2, len=2), Token(form='유승준', tag='NNP', start=5, len=3), Token(form='나오', tag='VV', start=8, len=2), Token(form='어서', tag='EC', start=9, len=2), Token(form='비추', tag='VV', start=12, len=2), Token(form='.', tag='SF', start=14, len=1)]


## 불용어

In [20]:
from kiwipiepy.utils import Stopwords
stopwords = Stopwords()
stopwords.stopwords  # set 자료형

{('ᆫ', 'ETM'),
 ('ᆫ', 'JX'),
 ('ᆫ다', 'EF'),
 ('ᆯ', 'ETM'),
 ('가', 'JKS'),
 ('같', 'VA'),
 ('것', 'NNB'),
 ('게', 'EC'),
 ('겠', 'EP'),
 ('고', 'EC'),
 ('고', 'JKQ'),
 ('과', 'JC'),
 ('과', 'JKB'),
 ('그', 'MM'),
 ('그', 'NP'),
 ('기', 'ETN'),
 ('까지', 'JX'),
 ('나', 'NP'),
 ('년', 'NNB'),
 ('는', 'ETM'),
 ('는', 'JX'),
 ('다', 'EC'),
 ('다', 'EF'),
 ('다고', 'EC'),
 ('다는', 'ETM'),
 ('대하', 'VV'),
 ('더', 'MAG'),
 ('던', 'ETM'),
 ('도', 'JX'),
 ('되', 'VV'),
 ('되', 'XSV'),
 ('들', 'XSN'),
 ('등', 'NNB'),
 ('따르', 'VV'),
 ('때', 'NNG'),
 ('때문', 'NNB'),
 ('라', 'EC'),
 ('라는', 'ETM'),
 ('로', 'JKB'),
 ('를', 'JKO'),
 ('만', 'JX'),
 ('만', 'NR'),
 ('말', 'NNG'),
 ('며', 'EC'),
 ('면', 'EC'),
 ('면서', 'EC'),
 ('명', 'NNB'),
 ('받', 'VV'),
 ('보', 'VV'),
 ('부터', 'JX'),
 ('사람', 'NNG'),
 ('성', 'XSN'),
 ('수', 'NNB'),
 ('아니', 'VCN'),
 ('않', 'VX'),
 ('어', 'EC'),
 ('어', 'EF'),
 ('어서', 'EC'),
 ('어야', 'EC'),
 ('없', 'VA'),
 ('었', 'EP'),
 ('에', 'JKB'),
 ('에게', 'JKB'),
 ('에서', 'JKB'),
 ('와', 'JC'),
 ('와', 'JKB'),
 ('우리', 'NP'),
 ('원', 'NNB'),


- 불용어 추가하기

In [21]:
stopwords.add(['길동','민수']) # 여러개 가능
stopwords.stopwords

{('ᆫ', 'ETM'),
 ('ᆫ', 'JX'),
 ('ᆫ다', 'EF'),
 ('ᆯ', 'ETM'),
 ('가', 'JKS'),
 ('같', 'VA'),
 ('것', 'NNB'),
 ('게', 'EC'),
 ('겠', 'EP'),
 ('고', 'EC'),
 ('고', 'JKQ'),
 ('과', 'JC'),
 ('과', 'JKB'),
 ('그', 'MM'),
 ('그', 'NP'),
 ('기', 'ETN'),
 ('길동', 'NNP'),
 ('까지', 'JX'),
 ('나', 'NP'),
 ('년', 'NNB'),
 ('는', 'ETM'),
 ('는', 'JX'),
 ('다', 'EC'),
 ('다', 'EF'),
 ('다고', 'EC'),
 ('다는', 'ETM'),
 ('대하', 'VV'),
 ('더', 'MAG'),
 ('던', 'ETM'),
 ('도', 'JX'),
 ('되', 'VV'),
 ('되', 'XSV'),
 ('들', 'XSN'),
 ('등', 'NNB'),
 ('따르', 'VV'),
 ('때', 'NNG'),
 ('때문', 'NNB'),
 ('라', 'EC'),
 ('라는', 'ETM'),
 ('로', 'JKB'),
 ('를', 'JKO'),
 ('만', 'JX'),
 ('만', 'NR'),
 ('말', 'NNG'),
 ('며', 'EC'),
 ('면', 'EC'),
 ('면서', 'EC'),
 ('명', 'NNB'),
 ('민수', 'NNP'),
 ('받', 'VV'),
 ('보', 'VV'),
 ('부터', 'JX'),
 ('사람', 'NNG'),
 ('성', 'XSN'),
 ('수', 'NNB'),
 ('아니', 'VCN'),
 ('않', 'VX'),
 ('어', 'EC'),
 ('어', 'EF'),
 ('어서', 'EC'),
 ('어야', 'EC'),
 ('없', 'VA'),
 ('었', 'EP'),
 ('에', 'JKB'),
 ('에게', 'JKB'),
 ('에서', 'JKB'),
 ('와', 'JC'),
 ('와', 'JKB')

In [22]:
stopwords.add(('철수','NNP')) # 품사지정도 가능 - 단 튜플로
stopwords.stopwords

{('ᆫ', 'ETM'),
 ('ᆫ', 'JX'),
 ('ᆫ다', 'EF'),
 ('ᆯ', 'ETM'),
 ('가', 'JKS'),
 ('같', 'VA'),
 ('것', 'NNB'),
 ('게', 'EC'),
 ('겠', 'EP'),
 ('고', 'EC'),
 ('고', 'JKQ'),
 ('과', 'JC'),
 ('과', 'JKB'),
 ('그', 'MM'),
 ('그', 'NP'),
 ('기', 'ETN'),
 ('길동', 'NNP'),
 ('까지', 'JX'),
 ('나', 'NP'),
 ('년', 'NNB'),
 ('는', 'ETM'),
 ('는', 'JX'),
 ('다', 'EC'),
 ('다', 'EF'),
 ('다고', 'EC'),
 ('다는', 'ETM'),
 ('대하', 'VV'),
 ('더', 'MAG'),
 ('던', 'ETM'),
 ('도', 'JX'),
 ('되', 'VV'),
 ('되', 'XSV'),
 ('들', 'XSN'),
 ('등', 'NNB'),
 ('따르', 'VV'),
 ('때', 'NNG'),
 ('때문', 'NNB'),
 ('라', 'EC'),
 ('라는', 'ETM'),
 ('로', 'JKB'),
 ('를', 'JKO'),
 ('만', 'JX'),
 ('만', 'NR'),
 ('말', 'NNG'),
 ('며', 'EC'),
 ('면', 'EC'),
 ('면서', 'EC'),
 ('명', 'NNB'),
 ('민수', 'NNP'),
 ('받', 'VV'),
 ('보', 'VV'),
 ('부터', 'JX'),
 ('사람', 'NNG'),
 ('성', 'XSN'),
 ('수', 'NNB'),
 ('아니', 'VCN'),
 ('않', 'VX'),
 ('어', 'EC'),
 ('어', 'EF'),
 ('어서', 'EC'),
 ('어야', 'EC'),
 ('없', 'VA'),
 ('었', 'EP'),
 ('에', 'JKB'),
 ('에게', 'JKB'),
 ('에서', 'JKB'),
 ('와', 'JC'),
 ('와', 'JKB')

- 불용어 삭제하기

In [23]:
stopwords.remove(['길동','철수','민수'])
stopwords.stopwords

{('ᆫ', 'ETM'),
 ('ᆫ', 'JX'),
 ('ᆫ다', 'EF'),
 ('ᆯ', 'ETM'),
 ('가', 'JKS'),
 ('같', 'VA'),
 ('것', 'NNB'),
 ('게', 'EC'),
 ('겠', 'EP'),
 ('고', 'EC'),
 ('고', 'JKQ'),
 ('과', 'JC'),
 ('과', 'JKB'),
 ('그', 'MM'),
 ('그', 'NP'),
 ('기', 'ETN'),
 ('까지', 'JX'),
 ('나', 'NP'),
 ('년', 'NNB'),
 ('는', 'ETM'),
 ('는', 'JX'),
 ('다', 'EC'),
 ('다', 'EF'),
 ('다고', 'EC'),
 ('다는', 'ETM'),
 ('대하', 'VV'),
 ('더', 'MAG'),
 ('던', 'ETM'),
 ('도', 'JX'),
 ('되', 'VV'),
 ('되', 'XSV'),
 ('들', 'XSN'),
 ('등', 'NNB'),
 ('따르', 'VV'),
 ('때', 'NNG'),
 ('때문', 'NNB'),
 ('라', 'EC'),
 ('라는', 'ETM'),
 ('로', 'JKB'),
 ('를', 'JKO'),
 ('만', 'JX'),
 ('만', 'NR'),
 ('말', 'NNG'),
 ('며', 'EC'),
 ('면', 'EC'),
 ('면서', 'EC'),
 ('명', 'NNB'),
 ('받', 'VV'),
 ('보', 'VV'),
 ('부터', 'JX'),
 ('사람', 'NNG'),
 ('성', 'XSN'),
 ('수', 'NNB'),
 ('아니', 'VCN'),
 ('않', 'VX'),
 ('어', 'EC'),
 ('어', 'EF'),
 ('어서', 'EC'),
 ('어야', 'EC'),
 ('없', 'VA'),
 ('었', 'EP'),
 ('에', 'JKB'),
 ('에게', 'JKB'),
 ('에서', 'JKB'),
 ('와', 'JC'),
 ('와', 'JKB'),
 ('우리', 'NP'),
 ('원', 'NNB'),


## 결합하기

In [24]:
text = train['review'][1]
text

'안봤지만 유승준나와서 비추.'

In [25]:
result = kiwi.tokenize(text)
result

[Token(form='안', tag='MAG', start=0, len=1),
 Token(form='보', tag='VV', start=1, len=1),
 Token(form='었', tag='EP', start=1, len=1),
 Token(form='지만', tag='EC', start=2, len=2),
 Token(form='유승준', tag='NNP', start=5, len=3),
 Token(form='나오', tag='VV', start=8, len=2),
 Token(form='어서', tag='EC', start=9, len=2),
 Token(form='비추', tag='VV', start=12, len=2),
 Token(form='.', tag='SF', start=14, len=1)]

In [26]:
# tokens = [t.form for t in result ]
# kiwi.join(tokens)
## kiwi.join()은 형태소와 품사의 쌍을 받도록 되어 있다.
## 따라서 Token 객체에서 형태소와 품사를 추출해 튜플로 만들어야 함

- 에러 수정

- kiwi.join(morphs)는 이 형태소와 품사를 바탕으로 원래 문장을 자연스럽게 합한다.

In [27]:
tokens = [ (t.form, t.tag) for t in result ]
kiwi.join(tokens)

'안 봤지만 유승준 나와서 비추.'

## 토큰화 해보기
- 불용어 제거와 함게 N, V로 시작하는 품사들만 토큰화 해보기

In [28]:
stopwords = Stopwords()

train_list = []
result = kiwi.tokenize(train['review'], stopwords = stopwords) #  불용어를 제외한 형태소만 추출
for tokens in tqdm(result, total = train['review'].shape[0]):
  tokens = [ t.form for t in tokens if t.tag[0] in 'NV']
  train_list.append(tokens)

  0%|          | 0/2000 [00:00<?, ?it/s]

In [29]:
len(train_list)

2000

In [30]:
min(len(tokens) for tokens in train_list)
# 0 이 있다는건 샘플을 버린게 있다는 의미!
# 해당 문장에서 유의미한 토큰이 하나도 추출되지 않았다는 의미

0

In [31]:
cnt = np.array([len(tokens) for tokens in train_list])
mask = cnt == 0
mask.sum()

49

In [32]:
train.loc[mask]

,id,review,target
31,train_31,대박....,1
102,train_102,What a great drama!!!,1
103,train_103,Space Jason!!!!,0
225,train_225,the roles play very real touching,1
307,train_307,참신하지는 않다,0
342,train_342,...,0
470,train_470,별로,0
524,train_524,harry potter go!,1
546,train_546,글쎄~ 별로던데~,0
581,train_581,ㅋㅋ,1


# spacy 를 이용한 형태소 분석
- 써보니까.. 별로다!!

In [33]:
!python -m spacy download ko_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.7/14.7 MB 30.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ko_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [34]:
import spacy

In [35]:
nip = spacy.load('ko_core_news_sm') # spacy가 모듈이라서 모듈함수 사용해야 된다고 하셨다...

In [36]:
text

'안봤지만 유승준나와서 비추.'

In [37]:
doc = nip(text)
doc

안봤지만 유승준나와서 비추.

In [38]:
doc[0].text, doc[0].lemma_, doc[0].tag_ # 실질적인 형태소 분석은 lemma 품사정보가 tag

('안봤지만', '안봤지+만', 'nq+jxt')

## 토큰화 해보기

In [39]:
# train_list = []
# for text in tqdm(train['review']):
#   doc = nip(text)
#   tmp_list = []
#   for tokens in doc :
#     tmp = tokens.lemma_.split('+')
#     tmp_list.extend(tmp)

#   train_list.append(tmp_list)

# konlpy 를 이용한 형태소 분석

- C++, 자바 등 다른 언어로 개발된 오픈소스 형태소 분석 라이브러리를 파이썬에서도 쉽게 사용할 수 있게 해주는 라이브러리
- 사용방법
  - 클래스 객체 생성
  - morphs 메서드와 pos 메서드를 사용하면 된다.
  - morphs 메서드
    - 토큰화
  - pos 메서드
    - 품사 태깅이 추가된 토큰화
-설치하기
  - pip install konlpy

In [40]:
%pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 41.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.8/493.8 kB 21.3 MB/s eta 0:00:00


In [41]:
from konlpy.tag import Okt, Komoran, Hannanum, Kkma # 모두가 앞이 대문자 - 클래스라는 의미

In [42]:
text

'안봤지만 유승준나와서 비추.'

## Okt

In [43]:
tokenizer = Okt()

In [44]:
tokenizer.morphs(text)

['안', '봤지만', '유승준', '나와서', '비추', '.']

In [45]:
tokenizer.pos(text) # 품사정보도 같이!!!!

[('안', 'VerbPrefix'),
 ('봤지만', 'Verb'),
 ('유승준', 'Noun'),
 ('나와서', 'Verb'),
 ('비추', 'Verb'),
 ('.', 'Punctuation')]

In [46]:
tokenizer = Komoran()
tokenizer.morphs(text)

['안', '보', '았', '지만', '유승준', '나', '와서', '비', '추', '.']

## Komoran

In [47]:
tokenizer = Komoran()
tokenizer.morphs(text)

['안', '보', '았', '지만', '유승준', '나', '와서', '비', '추', '.']

In [48]:
tokenizer.pos(text)

[('안', 'MAG'),
 ('보', 'VV'),
 ('았', 'EP'),
 ('지만', 'EC'),
 ('유승준', 'NNP'),
 ('나', 'JC'),
 ('와서', 'NNP'),
 ('비', 'XPN'),
 ('추', 'NNP'),
 ('.', 'SF')]

## Hannanum

In [49]:
tokenizer = Hannanum()

In [50]:
tokenizer.morphs(text)

['안봤지만', '유승준나와서', '비', '추', '.']

In [51]:
tokenizer.pos(text) # 품사정보도 같이!!!!

[('안봤지만', 'N'), ('유승준나와서', 'N'), ('비', 'X'), ('추', 'N'), ('.', 'S')]

##  Kkma(꼬꼬마)

In [52]:
tokenizer = Kkma()

In [53]:
tokenizer.morphs(text)

['안', '보', '았', '지만', '유', '승', '주', 'ㄴ', '나오', '아서', '비추', '.']

In [54]:
tokenizer.pos(text) # 품사정보도 같이!!!!

[('안', 'MAG'),
 ('보', 'VV'),
 ('았', 'EPT'),
 ('지만', 'ECE'),
 ('유', 'NNG'),
 ('승', 'NNG'),
 ('주', 'VV'),
 ('ㄴ', 'ETD'),
 ('나오', 'VV'),
 ('아서', 'ECD'),
 ('비추', 'NNG'),
 ('.', 'SF')]

# Mecab 형태소 분석기
- 빠름! 히히

In [55]:
%pip install python-mecab-ko

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 577.1/577.1 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 24.8 MB/s eta 0:00:00


In [56]:
from mecab import MeCab

In [57]:
tokenizer = MeCab()

In [58]:
tokenizer.morphs(text)

['안', '봤', '지만', '유승준', '나와서', '비추', '.']

In [59]:
tokenizer.pos(text) # 품사정보도 같이!!!!

[('안', 'MAG'),
 ('봤', 'VV+EP'),
 ('지만', 'EC'),
 ('유승준', 'NNP'),
 ('나와서', 'VV+EC'),
 ('비추', 'NNG'),
 ('.', 'SF')]

# 속도 비교 해보기

In [60]:
toikenizer = MeCab()
train_list = []
for text in tqdm(train['review']):
  tokens = tokenizer.pos(text)
  tokens = [t for t,p in tokens]
  train_list.append(tokens)

  0%|          | 0/2000 [00:00<?, ?it/s]

# Kiwi로 학습 데이터와 테스트 데이터를 만들기

```
kiwi를 이용하여 학습데이터와 테스트 데이터를 토큰화 해서 각각 train_list와 test_list에 담아주세요
```

- 내 코드

In [61]:
# stopwords = Stopwords()

# train_list = []
# result = kiwi.tokenize(train['review'], stopwords = stopwords)
# for tokens in tqdm(result, total = train['review'].shape[0]):
#   tokens = [ t.form for t in tokens if t.tag[0] in 'NV']
#   train_list.append(tokens)

In [62]:
# test_list = []
# result = kiwi.tokenize(test['review'], stopwords = stopwords)
# for tokens in tqdm(result, total = test['review'].shape[0]):
#   tokens = [ t.form for t in tokens if t.tag[0] in 'NV']
#   test_list.append(tokens)

- 강사님 코드

In [63]:
kiwi = Kiwi()
result = kiwi.tokenize(train['review']) # 여러 문장 토큰화 한 결과를 저장
train_list = []

# for tokens in tqdm(result):
#   tmp = [ t.form for t in tokens ]
#   train_list.append(tmp)

# 위의 for문이랑 똑같음
train_list = [ [ t.form for t in tokens ] for tokens in result]

In [64]:
kiwi = Kiwi()
result = kiwi.tokenize(test['review'])
test_list=[]
test_list = [ [ t.form for t in tokens ] for tokens in result]

- TF-IDF 변환
 - TF (단어 빈도수)
 - IDF (단어 중요도) - 단어가 문서에 적게 등장할수록 중요도 높음
 - 형태소 분석 후 문장을 숫자 벡터로 변환

In [65]:
train_list[:2]

[['이런', '최고', '의', '영화', '를', '이제서야', '보', '다니'],
 ['안', '보', '었', '지만', '유승준', '나오', '어서', '비추', '.']]

In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TfidfVectorizer 초기화
vec = TfidfVectorizer(max_features=500)

# train_list의 각 텍스트를 처리
train_data = vec.fit_transform(
    [' '.join(t) for t in train_list]  # train_list는 토큰화된 텍스트 리스트라고 가정
).A

In [67]:
train_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [68]:
test_data = vec.transform(
    [' '.join(t) for t in test_list]
).A

In [69]:
train_data.shape, test_data.shape

((2000, 500), (1000, 500))

In [70]:
(train_data.sum(axis=1)==0).sum()

131

In [82]:
target= train['target'].to_numpy().reshape(-1,1) # 이진 분류
target.shape

(2000, 1)

# 데이터셋클래스
- 데이터를 효율적으로 모델에 넣기 위해 만듬

In [72]:
import torch

In [73]:
class ReviewDataset(torch.utils.data.Dataset):
    def __init__(self, x, y=None):
        # x는 벡터화 된 문장데이터
        # y는 정답 레이블
        self.x , self.y = x, y

    # PyTorch의 DataLoader는 데이터셋의 길이를 바탕으로 배치(batch)를 나누기 때문에 필수 메서드
    def __len__(self):
        return self.x.shape[0]

    def __getitem__(self, i):
        item = {}
        item["x"] = torch.Tensor(self.x[i])
        if self.y is not None:
            item["y"] = torch.Tensor(self.y[i])
        return item

**3. Tensor vs tensor**

| 구분               | `torch.Tensor`            | `torch.tensor`          |
|--------------------|---------------------------|-------------------------|
| **종류**          | 클래스                    | 함수                   |
| **기본 dtype**     | `torch.float32` (기본값) | 입력 데이터에 따라 결정  |
| **데이터 보존**    | X (타입 명확하지 않음)     | O (타입과 값 유지)       |
| **사용 추천**      | 명시적으로 선언할 때      | 일반적으로 사용         |


In [74]:
dt = ReviewDataset(train_data,target)
dl= torch.utils.data.DataLoader(dt, batch_size=2)
batch = next(iter(dl))
batch

{'x': tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
          0

- 잔차 연결
  - x와 출력
𝑓
(
𝑥
)
의 크기(차원)가 같아야 𝑥+𝑓
(
𝑥
)
 연산이 가능

```
Input x
   │
   ├── Linear ── ReLU ── Dropout ── Linear
   │                                             │  
   └────────────── Skip Connection ┘
                │
               ReLU
                │
             Output

```

In [75]:
# 잔차 블록 구현
# 입력값에 추가적인 변환값을 더함
# 기울기 소실 문제 해결
# 입력 데이터 x: (batch_size, in_features) 인 텐서
class ResidualBlock(torch.nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.fx = torch.nn.Sequential(
            # 가중치 사용해서 입력 데이터 변형
            torch.nn.Linear(in_features, in_features), # 선형 변환 (Fully Connected Layer)
            # 양수값만 남기기, 음수 제거
            torch.nn.ReLU(), # 비선형성을 추가
            torch.nn.Dropout(0.5), # 노드 50% 비활성화
            torch.nn.Linear(in_features, in_features) # 선형 변환 (Fully Connected Layer)
        )
        # fx와 x를 더한 뒤, 그 결과물을 최종적으로 정리하기 위해 괄호 밖
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        fx = self.fx(x)
        hx = fx + x
        return self.relu(hx)

***Net 클래스***  
- 전체 신경망 모델을 구현합니다.
  - 초기 선형 레이어 + Batch Normalization
  - 여러 개의 ResidualBlock
  - 최종 출력 4개 클래스에 대한 출력 반환

In [91]:
class Net(torch.nn.Module):
    def __init__(self, in_features, n_layers=8):
        super().__init__()

        # Linear → BatchNorm → Activation 순서 지켜야 한다
        self.init_layer = torch.nn.Sequential(
            torch.nn.Linear(in_features, in_features // 2), # 선형변환 # 모델 복잡도 줄이기
            torch.nn.BatchNorm1d(in_features // 2), # 배치의 데이터 정규화해서 학습 안정화 # (batch_size, num_features)
            torch.nn.LeakyReLU() # 활성화 함수.. 왜 있어야하지 ?
        )
        res_list = [ ResidualBlock(in_features//2) for _ in range(n_layers) ]
        self.seq = torch.nn.Sequential(*res_list)
        self.output_layer = torch.nn.Linear(in_features//2,1)

    def forward(self, x):
        x = self.init_layer(x)
        x = self.seq(x)
        return self.output_layer(x)

In [92]:
Net(train_data.shape[1])(batch["x"])

tensor([[0.8320],
        [0.5021]], grad_fn=<AddmmBackward0>)

# 학습 loop 함수

In [93]:
def train_loop(dl, model, loss_fn, optimizer, device):
    epoch_loss = 0
    model.train()
    for batch in dl:
        pred = model(batch["x"].to(device))
        loss = loss_fn(pred, batch["y"].to(device))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    epoch_loss /= len(dl)
    return epoch_loss

# 예측 loop 함수

In [94]:
@torch.no_grad()
def test_loop(dl, model, loss_fn, device):
    epoch_loss = 0
    model.eval()

    act = torch.nn.Sigmoid()
    pred_list = []
    for batch in dl:
        pred = model( batch["x"].to(device) )
        if batch.get("y") is not None:
            loss = loss_fn(pred, batch["y"].to(device) )
            epoch_loss += loss.item() # item : 1개의 스칼라 값을 Python의 기본 데이터 타입(예: float)으로 변환

        pred = act(pred)
        pred = pred.to("cpu").numpy()
        pred_list.append(pred)

    pred = np.concatenate(pred_list)
    epoch_loss /= len(dl)
    return epoch_loss, pred

# 학습하기

In [101]:
n_splits = 5
batch_size = 32
epochs = 100
loss_fn = torch.nn.BCEWithLogitsLoss()

from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
cv = KFold(n_splits, shuffle=True, random_state=SEED)

In [105]:
is_holdout = True
reset_seeds(SEED)
score_list = []
for i, (tri, vai) in enumerate(cv.split(train_data)):
    # 학습 데이터
    train_dt = ReviewDataset(train_data[tri], target[tri])
    train_dl = torch.utils.data.DataLoader(train_dt, batch_size=batch_size, shuffle=True)
    # 검증 데이터
    valid_dt = ReviewDataset(train_data[vai], target[vai])
    valid_dl = torch.utils.data.DataLoader(valid_dt, batch_size=batch_size, shuffle=False)

    # 모델 객체 및 옵티마이저 생성
    model = Net(train_data.shape[1]).to(device)
    optimizer = torch.optim.Adam( model.parameters() )

    patience = 0 # 조기 종료 조건을 주기 위한 변수
    best_score = 0 # 현재 최고점수
    for _ in tqdm(range(epochs)):
        train_loss = train_loop(train_dl, model, loss_fn, optimizer, device)
        valid_loss, pred = test_loop(valid_dl, model, loss_fn, device)
        pred = np.where(pred > 0.5, 1, 0)
        # pred = np.argmax(pred, axis=1) -> 다중 분류
        score = accuracy_score(target[vai], pred)
        print(train_loss, valid_loss, score)

        patience += 1
        if score > best_score:
            best_score = score
            patience = 0
            torch.save( model.state_dict(), f"model{i}.pt" )

        if patience == 5:
            break

    score_list.append(best_score)
    print(f"ACC 최고점수: {best_score}")

    if is_holdout:
        break
print(score_list)
print(np.mean(score_list))

  0%|          | 0/100 [00:00<?, ?it/s]

0.6298381280899048 0.5930739182692307 0.705
0.3848632150888443 0.6007131842466501 0.6975
0.2593282461166382 0.6452617931824464 0.73
0.15703581169247627 0.8687189840353452 0.6975
0.1465930426120758 0.992966564802023 0.71
0.17486264631152154 0.9326218733420739 0.6975
0.14477642051875592 0.935110711134397 0.7175
0.12216436696238815 1.3279441732626696 0.69
ACC 최고점수: 0.73
[0.73]
0.73


In [106]:
print(score_list)
print(np.mean(score_list))

[0.73]
0.73


# 예측하기

In [107]:
test_dt = ReviewDataset(test_data)
test_dl = torch.utils.data.DataLoader(test_dt, shuffle=False, batch_size=batch_size)

In [109]:
pred_list = []
for i in range(n_splits):
    model = Net(train_data.shape[1]).to(device)
    state_dict = torch.load(f"model{0}.pt", weights_only=True)
    model.load_state_dict(state_dict)

    _, pred = test_loop(test_dl, model, None, device)
    pred_list.append(pred)

In [110]:
pred = np.mean(pred_list, axis=0)
pred = np.where(pred>0.5, 1, 0)
pred.shape

(1000, 1)